# MIDST Challenge
Black-box MIA on single table

# Import packages

In [1]:
import sys

sys.path.append("..")

from pathlib import Path

import pandas as pd
from clover.generators.findiff_generator import FinDiffGenerator

# 1.Configuration

In [2]:
# Paths
input_dir = Path("/data8/projets/dev_synthetic_data/data/MIDST/tabddpm_black_box/population/")
output_dir = input_dir / "synth"

# File
real_train = "real_train_no_id.csv"

# 2.Load real data

In [3]:
df_real_train = pd.read_csv(input_dir / real_train)

In [4]:
df_real_train.shape

(50000, 8)

# 3.Generate synthetic data
Generate 1st and 2nd generations synthetic data

In [5]:
# Metadata of the dataset

metadata = {
    "continuous": ["trans_date", "amount", "balance", "account"],
    "categorical": ["trans_type", "operation", "k_symbol", "bank"],
    "variable_to_predict": "trans_type",
}

In [9]:
# Train generator

gen = FinDiffGenerator(
    df=df_real_train,
    metadata=metadata,
    random_state=None,
    generator_filepath=None,
    learning_rate=1e-3,
    batch_size=256,
    diffusion_steps=500,
    epochs=500,
    mpl_layers=[1024, 1024, 1024, 1024],
    activation="lrelu",
    dim_t=64,
    cat_emb_dim=2,
    diff_beta_start_end=[1e-4, 0.02],
    scheduler="linear",
    epsilon=None,
)

gen.preprocess()
gen.fit(save_path=output_dir / "1st_gen")  # the path should exist

In [10]:
# Generate 1st geneartion synthetic data

df_synth_train = gen.sample(
    save_path=output_dir / "1st_gen",
    num_samples=len(df_real_train),
)

df_synth_test = gen.sample(
    save_path=output_dir / "1st_gen",
    num_samples=len(df_real_train)//2,
)

In [11]:
# Generate 2nd generation synthetic data

gen_2nd = FinDiffGenerator(
    df=df_synth_train,
    metadata=metadata,
    random_state=None,
    generator_filepath=None,
    learning_rate=1e-3,
    batch_size=256,
    diffusion_steps=500,
    epochs=500,
    mpl_layers=[1024, 1024, 1024, 1024],
    activation="lrelu",
    dim_t=64,
    cat_emb_dim=2,
    diff_beta_start_end=[1e-4, 0.02],
    scheduler="linear",
    epsilon=None,
)

gen_2nd.preprocess()
gen_2nd.fit(save_path=output_dir / "2nd_gen")  # the path should exist

df_synth_2nd = gen.sample(
    save_path=output_dir / "2nd_gen",
    num_samples=len(df_real_train)//2,
)